In [9]:
# check the size of earth, moon, and other images
# 149 earth, 148 moon, 149 for the rest so since there are 7other planets
# so from each, example earth, 149/7 = 21 images

# so the point of this script is to remove the background from the images provided
# the background is always black and the planets always circular so can
# find the centre of the circle and remove the background in each image provided

# earth images are in "Planets and Moons/Earth" they are .jpg
# moon images are in "Planets and Moons/Moon" they are .jpg
# other planets are in "Planets and Moons/[other]" they are .png, there other is
# Jupiter, Mars, Mercury, Neptune, Pluto, Uranus, Venus

import os
import cv2
import numpy as np

def remove_background_and_save(images_dir, flag=0):
    # Get a list of all files in the specified directory
    files = os.listdir(images_dir)
    
    counter = 0
    # Iterate over each file
    for file in files:
        counter += 1

        # if counter is greater than 21 and flag is 1, break
        if counter > 21 and flag == 1:
            break

        # Load the image
        img_path = os.path.join(images_dir, file)
        img = cv2.imread(img_path)
        
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Apply GaussianBlur to reduce noise
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # There is only 1 circle in the image so use HoughCircles
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
                                   param1=50, param2=30, minRadius=0, maxRadius=0)

        # There will always be a circle in the image, and only 1 circle

        # so extract the circle from the image and save it

        # Get the circle
        circle = circles[0][0]
        x, y, r = circle
        # convert to int
        x, y, r = int(x), int(y), int(r)

        # Create a mask
        mask = np.zeros_like(gray)
        cv2.circle(mask, (x, y), r, 255, -1)

        # Apply the mask
        # Apply the mask to the original image
        masked_img = cv2.bitwise_and(img, img, mask=mask)

        # Create a mask for the background
        inverted_mask = cv2.bitwise_not(mask)
        background = np.full_like(img, (0, 0, 0))
        background_masked = cv2.bitwise_and(background, background, mask=inverted_mask)

        # Combine the masked circular image with the background
        final_img = masked_img + background_masked

        # save the circular images in "Data/" folder with the name of the file being the planet it represents and the number (which is the default image name anyways) 
        planet_name = images_dir.split("/")[-1]
        save_dir = "Data/" + planet_name
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        save_path = os.path.join(save_dir, file)
        cv2.imwrite(save_path, final_img)



# Usage example
earth_dir = "planets/Planets/Earth"
moon_dir = "planets/Planets/Moon"
# same but for now Jupiter, Mars, Mercury, Neptune, Pluto, Uranus, Venus
ju_dir = "planets/Planets/Jupiter"
mars_dir = "planets/Planets/Mars"
mercury_dir = "planets/Planets/Mercury"
neptune_dir = "planets/Planets/Neptune"
pluto_dir = "planets/Planets/Pluto"
uranus_dir = "planets/Planets/Uranus"
venus_dir = "planets/Planets/Venus"


remove_background_and_save(earth_dir)
remove_background_and_save(moon_dir)
# remove for others with flag = 1
remove_background_and_save(ju_dir, 1)
remove_background_and_save(mars_dir, 1)
remove_background_and_save(mercury_dir, 1)
remove_background_and_save(neptune_dir, 1)
remove_background_and_save(pluto_dir, 1)
remove_background_and_save(uranus_dir, 1)
remove_background_and_save(venus_dir, 1)


In [11]:
class Planet:
    def __init__(self, x, y, r):
        # x and y are the coordinates of the center of the circle
        self.x = x
        self.y = y
        self.r = r
        self.fileName = ""
        self.type = ""
    
    # create getter methods for the x, y, and r attributes
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def getR(self):
        return self.r
    
    def setFileName(self, fileName):
        self.fileName = fileName
    
    def setType(self, type):
        self.type = type

In [15]:
# Separate planets from background
# Save the image with the planets separated from the background
# into folder called separatedPlanets[time]

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
    

def separate_planet(image_path, save_path, show=1):
    # Load the image
    # Replace with the actual path to your image
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # show the image
    cv2.imshow("Original Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

        
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and help circle detection
    gray_blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # Use Hough Circle Transform to detect circles
    circles = cv2.HoughCircles(
        gray_blurred,
        cv2.HOUGH_GRADIENT,
        dp=1,  # inverse ratio of the accumulator resolution
        minDist=100,  # minimum distance between the centers of detected circles
        param1=20,  # gradient value for edge detection
        param2=20,  # accumulator threshold for circle detection
        minRadius=60,  # minimum radius of the detected circle
        maxRadius=100,  # maximum radius of the detected circle
    )

    listOfPlanetObjects = []

    # If circles are found, draw them on the image
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # draw the outer circle
            cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
            # draw the center of the circle
            cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 3)
            # Need to create a new Planet object and save it to the list
            planet = Planet(i[0], i[1], i[2])
            planet.setFileName(fromPathGetFileName(image_path))
            listOfPlanetObjects.append(planet)
        
    else:
        print("No circles detected.")

    return listOfPlanetObjects

def fromPathGetFileName(path):
    return os.path.basename(path)


def cropImage(img):
        # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find contours in the image
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the maximum area (assuming it represents the planet)
    max_contour = max(contours, key=cv2.contourArea)

    # Create a mask for the planet using the maximum contour
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [max_contour], 0, 255, thickness=cv2.FILLED)

    # Bitwise AND operation to apply the mask to the original image
    result = cv2.bitwise_and(img, img, mask=mask)

    # Find the bounding box of the planet
    x, y, w, h = cv2.boundingRect(max_contour)

    # Crop the image to the bounding box
    cropped_image = result[y:y+h, x:x+w]

    # Resize the cropped image to your desired dimensions
    desired_size = (300, 300)  # specify the dimensions you want
    resized_image = cv2.resize(cropped_image, desired_size)


    return resized_image

# now a generic function
# it will, take the image path, the save path, and the planet object
# it will try to detect the planet in the image
# then it will remove the background and save the image to the save path
def planetDetection(input_path, output_path, flag=0):

    files = os.listdir(input_path)
    
    counter_outer = 0
    # Iterate over each file
    for file in files:
        counter_outer += 1

        # if counter is greater than 21 and flag is 1, break
        if counter_outer > 21 and flag == 1:
            break

        
        file_path = os.path.join(input_path, file)

        listOfPlanetObjects = separate_planet(file_path, output_path, 1)

        counter = -1
        for planet in listOfPlanetObjects:
            counter += 1
            # read the image
            img = cv2.imread(output_path, cv2.IMREAD_COLOR)
            # create a mask
            mask = np.zeros(img.shape[:2], np.uint8)
            cv2.circle(mask, (planet.x, planet.y), planet.r, (255, 255, 255), -1)
            # apply the mask
            result = cv2.bitwise_and(img, img, mask=mask)
            # crop it
            result = cropImage(result)
            # save the image
            # change the output path to include a counter so that the images are not overwritten
            # it should look like Planets/[name of the image]_[counter].jpg
            # save the circular images in "Data/" folder with the name of the file being the planet it represents and the number (which is the default image name anyways) 
            planet_name = input_path.split("/")[-1]
            save_dir = "Data/" + planet_name
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            save_path = os.path.join(save_dir, file)
            cv2.imwrite(save_path, result)

        return listOfPlanetObjects



# Usage example
earth_dir = "planets/Planets/Earth"
moon_dir = "planets/Planets/Moon"
# same but for now Jupiter, Mars, Mercury, Neptune, Pluto, Uranus, Venus
ju_dir = "planets/Planets/Jupiter"
mars_dir = "planets/Planets/Mars"
mercury_dir = "planets/Planets/Mercury"
neptune_dir = "planets/Planets/Neptune"
pluto_dir = "planets/Planets/Pluto"
uranus_dir = "planets/Planets/Uranus"
venus_dir = "planets/Planets/Venus"

# save path
save = "Data"


# use planetDectection on all input paths, same output path
planetDetection(earth_dir, save)
planetDetection(moon_dir, save)
planetDetection(ju_dir, save, 1)
planetDetection(mars_dir, save, 1)
planetDetection(mercury_dir, save,1)
planetDetection(neptune_dir, save, 1)
planetDetection(pluto_dir, save, 1)
planetDetection(uranus_dir, save, 1)
planetDetection(venus_dir, save, 1)


AttributeError: 'NoneType' object has no attribute 'shape'

: 